# Example of SasRec training/inference with Parquet Module

## Imports and session initialization

In [1]:
import copy

import torch
import lightning as L
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.data import (
    FeatureHint,
    FeatureSource,
    FeatureType,
)
from replay.data.nn import (
    TensorFeatureInfo,
    TensorFeatureSource,
    TensorSchema,
)
from replay.metrics import MAP, OfflineMetrics, Precision, Recall
from replay.splitters import LastNSplitter, RatioSplitter
from replay.utils.session_handler import get_spark_session

# Fix seed to ensure reproducibility
L.seed_everything(42)

import warnings
warnings.filterwarnings("ignore")

Seed set to 42


In [2]:
spark_session = get_spark_session()

26/01/14 11:41:26 WARN Utils: Your hostname, ecs-evtsinovnik-64 resolves to a loopback address: 127.0.1.1; using 10.11.12.49 instead (on interface eth0)
26/01/14 11:41:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/14 11:41:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/14 11:41:27 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


## Preparing data
In this example, we will be using the MovieLens dataset, namely the 1m subset.  
Begin by loading interactions, item features and user features using the created session.

---
**NOTE**

Current implementation of SasRec handles only item and interactions features. It does not take into account user features. As such, they are only used in this example to get complete lists of users.

---

In [3]:
schema = T.StructType([
    T.StructField("user_id", T.IntegerType(), True),
    T.StructField("item_id", T.IntegerType(), True),
    T.StructField("rating", T.IntegerType(), True),
    T.StructField("timestamp", T.LongType(), True),
])
interactions = spark_session.read.schema(schema).option("sep", "\t").csv("data/ml1m_ratings.dat")

# NOTE: The following code block is optional and is used
# to counteract the issue of identical timestamps in the dataset.
# Uncomment if you wish to use it.
# int_win = Window.partitionBy("user_id").orderBy("item_id")
# interactions = (
#     interactions.select(["user_id", "item_id", "timestamp"])
#     .withColumn("ts", F.col("timestamp").cast("long") * 1000)
#     .withColumn("row_num", F.row_number().over(int_win))
#     .withColumn("timestamp", (F.col("ts") + F.col("row_num")).cast("string"))
#     .drop("ts", "row_num")
# )
interactions.show(n=5)

26/01/14 11:41:28 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
26/01/14 11:41:28 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
26/01/14 11:41:28 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|   1193|     5|978300760|
|      1|    661|     3|978302109|
|      1|    914|     3|978301968|
|      1|   3408|     4|978300275|
|      1|   2355|     5|978824291|
+-------+-------+------+---------+
only showing top 5 rows



In [4]:
schema = T.StructType([
    T.StructField("user_id", T.IntegerType(), True),
    T.StructField("age", T.IntegerType(), True),
    T.StructField("gender", T.StringType(), True),
    T.StructField("occupation", T.StringType(), True),
    T.StructField("zip_code", T.LongType(), True),
])
user_features = spark_session.read.schema(schema).option("sep", "\t").csv("data/ml1m_users.dat")
user_features.show(n=5)

+-------+----+------+----------+--------+
|user_id| age|gender|occupation|zip_code|
+-------+----+------+----------+--------+
|      1|null|     1|        10|   48067|
|      2|null|    56|        16|   70072|
|      3|null|    25|        15|   55117|
|      4|null|    45|         7|    2460|
|      5|null|    25|        20|   55455|
+-------+----+------+----------+--------+
only showing top 5 rows



In [ ]:
schema = T.StructType([
    T.StructField("item_id", T.IntegerType(), True),
    T.StructField("movie_title", T.StringType(), True),
    T.StructField("genre", T.StringType(), True),
])
item_features = spark_session.read.schema(schema).option("sep", "\t").csv("data/ml1m_items.dat")
item_features.show(n=5)

+-------+--------------------+--------------------+
|item_id|         movie_title|               genre|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



### Encode catagorical data.
To ensure all categorical data is fit for training, it needs to be encoded using the `LabelEncoder` class. Create an instance of the encoder, providing a `LabelEncodingRule` for each categorcial column in the dataset.

In [6]:
from replay.preprocessing.label_encoder import LabelEncoder, LabelEncodingRule
from replay.utils.types import SparkDataFrame


def encode_data(
    queries: SparkDataFrame, items: SparkDataFrame, interactions: SparkDataFrame, label_encoder: LabelEncoder
):
    full_data = interactions.join(queries, on="user_id").join(items, on="item_id")
    full_data = label_encoder.fit_transform(full_data)

    return full_data

In [7]:
encoder = LabelEncoder(
    [
        LabelEncodingRule("user_id", default_value="last"),
        LabelEncodingRule("item_id", default_value="last"),
        LabelEncodingRule("genre", default_value="last"),
    ]
)
encoded_interactions = encode_data(user_features, item_features, interactions, encoder)

26/01/14 11:41:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/14 11:41:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/14 11:41:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/14 11:41:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/14 11:41:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/14 11:41:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/14 1

### Split interactions into the train, validation and test datasets using RatioSplitter

In order to facilitate the model's training, we split the dataset in the following way:
1) A 60/40 data split of original data for training and subsequent splits
2) A 75/25 split of the leftover data for testing/validation respectively (i.e. 30%/10% of the full dataset)

We also remove cold users/items after each split.

In [8]:
train_events, test_events = RatioSplitter(
    test_size=0.4,
    divide_column="user_id",
    query_column="user_id",
    timestamp_column="timestamp",
    drop_cold_users=True,
    drop_cold_items=True,
).split(encoded_interactions)

print(f"{train_events.count()=}, {test_events.count()=}")

train_events.count()=597866, test_events.count()=402190


In [9]:
test_events, val_events = RatioSplitter(
    test_size=0.25,
    divide_column="user_id",
    query_column="user_id",
    timestamp_column="timestamp",
    drop_cold_users=True,
    drop_cold_items=True,
).split(test_events)

print(f"{test_events.count()=}, {val_events.count()=}")

test_events.count()=299479, val_events.count()=102645


### Split the validation dataset into events and ground_truth

For both validation and testing data, the last N items are split into ground truth, which will be used to calculate metrics.

In [10]:
VALIDATION_GROUND_TRUTH_INTERACTIONS_PER_USER = 3
TEST_GROUND_TRUTH_INTERACTIONS_PER_USER = 3

val_events, val_gt = LastNSplitter(
    N=VALIDATION_GROUND_TRUTH_INTERACTIONS_PER_USER,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
).split(val_events)
print(f"{val_events.count()=}, {val_gt.count()=}")

test_events, test_gt = LastNSplitter(
    N=TEST_GROUND_TRUTH_INTERACTIONS_PER_USER, divide_column="user_id", query_column="user_id", strategy="interactions"
).split(test_events)
print(f"{test_events.count()=}, {test_gt.count()=}")

val_events.count()=84611, val_gt.count()=18034


test_events.count()=281359, test_gt.count()=18120


### Dataset preprocessing ("baking")
SasRec expects each user in the batch to provide their events in form of a sequence. For this reason, the event splits must be properly processed using the `groupby_sequences` function provided by RePlay.

In [11]:
from replay.data.nn.utils import groupby_sequences


def bake_data(full_data: SparkDataFrame):
    grouped_interactions = groupby_sequences(events=full_data, groupby_col="user_id", sort_col="timestamp")

    return grouped_interactions

In [12]:
train_events = bake_data(train_events)
val_events = bake_data(val_events)
val_gt = bake_data(val_gt)
test_events = bake_data(test_events)

To ensure we don't validate on unknown users, we join train and validation data by user ids, leaving only the common ones.  
We also pre-package the validation data with its ground truth and train-time events.

In [13]:
# Keep common query ids between val_dataset and val_gt
val_events = val_events.join(val_gt, on="user_id", how="left_semi")
val_gt = val_gt.join(val_events, on="user_id", how="left_semi")

gt_to_join = val_gt.select(["user_id", "item_id"]).withColumnRenamed("item_id", "ground_truth")
train_to_join = train_events.select(["user_id", "item_id"]).withColumnRenamed("item_id", "train")

val_events = val_events.join(gt_to_join, on="user_id", how="left")
val_events = val_events.join(train_to_join, on="user_id", how="left")

TRAIN_LEN = val_events.select(F.max(F.size("train")).alias("res")).collect()[0].res
GT_LEN = val_events.select(F.max(F.size("ground_truth")).alias("res")).collect()[0].res

In [14]:
TRAIN_PATH = "temp/data/train.parquet"
VAL_PATH = "temp/data/val.parquet"
TEST_PATH = "temp/data/test.parquet"

train_events.write.mode("overwrite").parquet(TRAIN_PATH)
val_events.write.mode("overwrite").parquet(VAL_PATH)
test_events.write.mode("overwrite").parquet(TEST_PATH)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model. It is required by the SasRec model to correctly create embeddings at train time.

In [15]:
EMBEDDING_DIM = 64

ITEM_FEATURE_NAME = "item_id"
NUM_UNIQUE_ITEMS = len(encoder.mapping["item_id"])
NUM_UNIQUE_CLASS_VALUES = len(encoder.mapping["genre"])

tensor_schema = TensorSchema(
    [
        TensorFeatureInfo(
            name="item_id",
            is_seq=True,
            padding_value=NUM_UNIQUE_ITEMS,
            cardinality=NUM_UNIQUE_ITEMS + 1,  # taking into account padding
            embedding_dim=EMBEDDING_DIM,
            feature_type=FeatureType.CATEGORICAL,
            feature_sources=[TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_id")],
            feature_hint=FeatureHint.ITEM_ID,
        ),
        TensorFeatureInfo(
            name="genre",
            is_seq=True,
            padding_value=NUM_UNIQUE_CLASS_VALUES,
            cardinality=NUM_UNIQUE_CLASS_VALUES + 1,  # taking into account padding
            embedding_dim=EMBEDDING_DIM,
            feature_type=FeatureType.CATEGORICAL,
            feature_sources=[TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_id")],
        ),
    ]
)

### Configure ParquetModule and transformation pipelines
The `ParquetModule` class enables training of models on large datasets by reading data in streaming mode. This class initialized with a metadata dict containing information about dataset's features and miscellanious options for initialization (such as shuffling).

Additionally, `ParquetModule` supports "transform pipelines" - stage-specific modules implementing additional preprocessing to be performed on batch level right before the forward pass.  

In our case, we create the following pipelines:
1) Training:
    1. Create a target, which contains the shifted item sequence that represents the next item in the sequence (for the next item prediction task).
    2. Optionally sample negatives (required only for sampled losses).
    3. Rename features to match it with expected format by the model during training.
    4. Unsqueeze target (`positive_labels`) and it's padding mask (`target_padding_mask`) for getting required shape of this tensors for loss computation.
    5. Group input features to be embed in expected format.

2) Validation/Inference:
    1. Rename/group features to match it with expected format by the model during valdiation/inference.

Then, metadata for ParquetModule should be created. It contains shape and padding value for each feature.

In [16]:
from replay.nn.transforms import (
    UnsqueezeTransform,
    GroupTransform,
    RenameTransform,
    NextTokenTransform,
    UniformNegativeSamplingTransform,
)

MAX_SEQ_LEN = 50
BATCH_SIZE = 32
SHIFT = 1

TRANSFORMS = {
    "train": [
        NextTokenTransform(
            label_field="item_id", query_features="user_id", shift=SHIFT, out_feature_name="positive_labels"
        ),
        UniformNegativeSamplingTransform(vocab_size=NUM_UNIQUE_ITEMS, num_negative_samples=200),
        RenameTransform(
            {"user_id": "query_id", "item_id_mask": "padding_mask", "positive_labels_mask": "target_padding_mask"}
        ),
        UnsqueezeTransform("target_padding_mask", -1),
        UnsqueezeTransform("positive_labels", -1),
        GroupTransform({"feature_tensors": ["item_id", "genre"]}),
    ],
    "val": [
        RenameTransform({"user_id": "query_id", "item_id_mask": "padding_mask"}),
        GroupTransform({"feature_tensors": ["item_id", "genre"]}),
    ],
    "test": [
        RenameTransform({"user_id": "query_id", "item_id_mask": "padding_mask"}),
        GroupTransform({"feature_tensors": ["item_id", "genre"]}),
    ],
}

def create_meta(seq_len: int, add_gt: bool = False):
    meta = {
        "user_id": {},
        "item_id": {"shape": seq_len, "padding": tensor_schema["item_id"].padding_value},
        "genre": {"shape": seq_len, "padding": tensor_schema["genre"].padding_value},
    }
    if add_gt:
        meta.update({"train": {
            "shape": seq_len, "padding": tensor_schema["item_id"].padding_value,
        },
        "ground_truth": {
            "shape": seq_len, "padding": tensor_schema["item_id"].padding_value,
        }
        })
    return meta

METADATA = {
    "train": create_meta(MAX_SEQ_LEN+1),
    "val": create_meta(MAX_SEQ_LEN, add_gt=True),
    "test": create_meta(MAX_SEQ_LEN),
}

In [17]:
from replay.data.nn import ParquetModule

parquet_module = ParquetModule(
    train_path=TRAIN_PATH,
    val_path=VAL_PATH,
    test_path=TEST_PATH,
    batch_size=BATCH_SIZE,
    metadata=METADATA,
    transforms=TRANSFORMS,
)

**NOTE**: 
You can also create a module specifically for training/inference by providing only their respective datapaths.
In such cases it's possible to pass to ParquetModule either all transforms or transforms for used data splits only.

For example:

In [18]:
parquet_module_train_val = ParquetModule(
    train_path=TRAIN_PATH,
    val_path=VAL_PATH,
    batch_size=BATCH_SIZE,
    metadata=METADATA,
    transforms=TRANSFORMS
)

## Train model
### Create SasRec model instance and run the training stage using lightning
We may now train the model using the Lightning trainer class. 

RePlay's implementation of SasRec is designed in a modular, **block-based approach**. Instead of passing configuration parameters to the constructor, SasRec is now built by providing fully initialized components that makes the model more flexible and easier to extend. SasRec consists of the body and loss. Body consits of the following components: embedder, aggregator, encoder, mask, output_normalization, loss.

#### Components of SasRec
* `Body` - The body component defines the full model excluding loss.
* `Loss` - The loss component defines how the training loss is computed. All available loss implementations are located in nn/loss.

#### Components of SasRecBody

* `Embedder` -The embedder is responsible for converting input features into embeddings. The default implementation is `SequenceEmbedding`, which supports the following feature types: categorical, categorical_list, numerical, numerical_list

* `Aggregator` - The aggregator combines all embeddings produced by the embedder and adds positional embeddings.
Currently, `SasRecAggregator` is supported. It internally uses one of the following embedding aggregation strategies: `SumAggregator`, `ConcatAggregator`.

* `Encoder` - The encoder represents the core transformer block of the model. The following implementations are currently available: `SasRecTransformerLayer` (default one), `DiffAttentionLayer` (a modified version with differential attention).

* `Mask` - The mask is an object that creates attention mask by input. RePlay supports `DefaultAttentionMask` creating a lower-triangular attention mask.

* `Output Normalization` - Any suitable PyTorch normalization layer may be used as output_normalization, for example: torch.nn.LayerNorm or torch.nn.RMSNorm

In [19]:
from replay.nn import DefaultAttentionMask, SequenceEmbedding, SumAggregator
from replay.nn.loss import CESampled
from replay.nn.sequential import SasRec, SasRecBody, PositionAwareAggregator, SasRecTransformerLayer


NUM_BLOCKS = 1
NUM_HEADS = 1
DROPOUT = 0.0

body = SasRecBody(
    embedder=SequenceEmbedding(
        schema=tensor_schema,
        categorical_list_feature_aggregation_method="sum",
    ),
    embedding_aggregator=PositionAwareAggregator(
        embedding_aggregator=SumAggregator(embedding_dim=EMBEDDING_DIM),
        max_sequence_length=MAX_SEQ_LEN,
        dropout=DROPOUT,
    ),
    attn_mask_builder=DefaultAttentionMask(
        reference_feature_name=tensor_schema.item_id_feature_name,
        num_heads=NUM_HEADS,
    ),
    encoder=SasRecTransformerLayer(
        embedding_dim=EMBEDDING_DIM,
        num_heads=NUM_HEADS,
        num_blocks=NUM_BLOCKS,
        dropout=DROPOUT,
        activation="relu",
    ),
    output_normalization=torch.nn.LayerNorm(EMBEDDING_DIM),
)
sasrec = SasRec(
    body=body,
    loss=CESampled(padding_idx=tensor_schema.item_id_features.item().padding_value),
)

#### Default Configuration

Default SasRec model may be created quickly via method *from_params*. Default model instance has CE loss, original SasRec transformer layes, and embeddings are aggregated via sum.

In [20]:
default_sasrec = SasRec.from_params(
    schema=tensor_schema, 
    embedding_dim=EMBEDDING_DIM, 
    max_sequence_length=MAX_SEQ_LEN,
    num_heads=NUM_HEADS,
    num_blocks=NUM_BLOCKS,
    dropout=DROPOUT,
    excluded_features=None
    )

A universal PyTorch Lightning module is provided that can work with any RePlay NN model.

In [21]:
from replay.nn.lightning import LightningModule
from replay.models.nn.optimizer_utils import FatOptimizerFactory, FatLRSchedulerFactory

model = LightningModule(
    sasrec,
    optimizer_factory=FatOptimizerFactory(),
    lr_scheduler_factory=FatLRSchedulerFactory(),
)

To facilitate training, we add the following callbacks:
1) `ModelCheckpoint` - to save the best trained model based on its Recall metric. It's a default Lightning Callback.
1) `ComputeMetricsCallback` - to display a detailed validation metric matrix after each epoch. It's a custom RePlay callback for computing recsys metrics on validation. It supports model's logits postpocessing (before metrics computing), we will use RePlay `SeenItemsFilter` in order to compute metrics on unseen ground truth items only.


In [22]:
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger
from replay.nn.lightning.callbacks import ComputeMetricsCallback
from replay.nn.lightning.postprocessors import SeenItemsFilter

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    monitor="recall@10",
    mode="max",
)

postprocessors = [
    SeenItemsFilter(
        seen_path=VAL_PATH,
        item_count=NUM_UNIQUE_ITEMS,
        query_column="user_id",
        item_column=tensor_schema.item_id_feature_name,
    )
]
validation_metrics_callback = ComputeMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=NUM_UNIQUE_ITEMS,
    postprocessors=postprocessors
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SasRec-example")

trainer = L.Trainer(
    max_epochs=5,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

trainer.fit(model, datamodule=parquet_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | SasRec | 285 K  | train
-----------------------------------------
285 K     Trainable params
0         Non-trainable params
285 K     Total params
1.140     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

b=0, d=0
b=1, d=0


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

b=157, d=0
b=158, d=0
b=159, d=0
b=160, d=0
b=161, d=0
b=162, d=0
b=163, d=0
log_dict
k              1        10        20         5
map     0.039962  0.010613  0.006150  0.017481
ndcg    0.039962  0.030628  0.025848  0.035006
recall  0.000799  0.005629  0.009025  0.003380



Validation: |          | 0/? [00:00<?, ?it/s]

b=0, d=0
b=1, d=0
b=2, d=0
b=3, d=0
b=4, d=0
b=5, d=0
b=6, d=0
b=7, d=0
b=8, d=0
b=9, d=0
b=10, d=0
b=11, d=0
b=12, d=0
b=13, d=0
b=14, d=0
b=15, d=0
b=16, d=0
b=17, d=0
b=18, d=0
b=19, d=0
b=20, d=0
b=21, d=0
b=22, d=0
b=23, d=0
b=24, d=0
b=25, d=0
b=26, d=0
b=27, d=0
b=28, d=0
b=29, d=0
b=30, d=0
b=31, d=0
b=32, d=0
b=33, d=0
b=34, d=0
b=35, d=0
b=36, d=0
b=37, d=0
b=38, d=0
b=39, d=0
b=40, d=0
b=41, d=0
b=42, d=0
b=43, d=0
b=44, d=0
b=45, d=0
b=46, d=0
b=47, d=0
b=48, d=0
b=49, d=0
b=50, d=0
b=51, d=0
b=52, d=0
b=53, d=0
b=54, d=0
b=55, d=0
b=56, d=0
b=57, d=0
b=58, d=0
b=59, d=0
b=60, d=0
b=61, d=0
b=62, d=0
b=63, d=0
b=64, d=0
b=65, d=0
b=66, d=0
b=67, d=0
b=68, d=0
b=69, d=0
b=70, d=0
b=71, d=0
b=72, d=0
b=73, d=0
b=74, d=0
b=75, d=0
b=76, d=0
b=77, d=0
b=78, d=0
b=79, d=0
b=80, d=0
b=81, d=0
b=82, d=0
b=83, d=0
b=84, d=0
b=85, d=0
b=86, d=0
b=87, d=0
b=88, d=0
b=89, d=0
b=90, d=0
b=91, d=0
b=92, d=0
b=93, d=0
b=94, d=0
b=95, d=0
b=96, d=0
b=97, d=0
b=98, d=0
b=99, d=0
b=100, d=0

Epoch 4, global step 945: 'recall@10' reached 0.00614 (best 0.00614), saving model to '/home/evtsinovnik/replay/examples/.checkpoints/epoch=4-step=945-v5.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


b=160, d=0
b=161, d=0
b=162, d=0
b=163, d=0
log_dict
k              1        10        20         5
map     0.042447  0.011350  0.006680  0.018105
ndcg    0.042447  0.032961  0.028402  0.036359
recall  0.000849  0.006138  0.010138  0.003491



We can now laod the best model using the path stored in the callback.

In [23]:
best_model = LightningModule.load_from_checkpoint(checkpoint_callback.best_model_path, model=sasrec)

## Inference stage

### Run inference
We can now perform inference using the data module we created earlier. Recommendations can be fetched in four formats: PySpark DataFrame, Pandas DataFrame, Polars DataFrame or raw PyTorch tensors. Each of the types corresponds a callback. Inthis example, we'll be using the `PandasTopItemsCallback`.
Prediction callbacks also can filter results using postprocessors.

In [24]:
from replay.nn.lightning.callbacks import PandasTopItemsCallback

csv_logger = CSVLogger(save_dir=".logs/test", name="SasRec-example")

TOPK = [1, 5, 10, 20]

postprocessors = [
    SeenItemsFilter(
        seen_path=TEST_PATH,
        item_count=NUM_UNIQUE_ITEMS,
        query_column="user_id",
        item_column=tensor_schema.item_id_feature_name,
    )
]

pandas_prediction_callback = PandasTopItemsCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

trainer = L.Trainer(callbacks=[pandas_prediction_callback], logger=csv_logger, inference_mode=True)

trainer.predict(best_model, datamodule=parquet_module, return_predictions=False)

pandas_res = pandas_prediction_callback.get_result()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [25]:
pandas_res

,user_id,item_id,score
0,3,1199,6.612652
0,3,1130,6.487318
0,3,1820,6.332965
0,3,1275,6.253757
0,3,971,6.167967
...,...,...,...
6039,6025,3107,5.192743
6039,6025,1830,5.170015
6039,6025,2374,5.094969
6039,6025,107,5.087105


### Calculating metrics

*test_gt* is already encoded, so we can use it for computing metrics.

In [26]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK)], query_column="user_id", rating_column="score"
)(pandas_res, test_gt.toPandas())

In [27]:
metrics_to_df(result_metrics)

k,1,10,20,5
MAP,0.058278,0.055889,0.065564,0.044543
Precision,0.058278,0.044487,0.035844,0.051358
Recall,0.019426,0.148289,0.238962,0.085596
